In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np

# requires installation with: pip install pycountry
import pycountry

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
'''
Load & format OECD International Migration Database data
'''

def load_oecd_data():
    """ 
    Load oecd data file
    Reshape the dataset to have country, destination, year, asylum_seekers and migrants columns
    Set hierarchical index (country, year)
    
    Returns
    --------
    oecd: data frame containing oecd data
    """
    #load dataset
    oecd = pd.read_csv('data/oecd_data.csv',na_values=['..'])
    #reshape table
    oecd.set_index(['country','destination', 'year','variable'], inplace=True)
    oecd=oecd.unstack()
    oecd.columns = oecd.columns.droplevel(0)
    #drop unwanted columns
    oecd.reset_index(drop=False, inplace=True)
    #rename columns
    oecd = oecd[['country','destination','year','Inflows of asylum seekers by nationality','Inflows of foreign population by nationality']]
    oecd = oecd.rename(columns={'Inflows of asylum seekers by nationality': 'asylum_seekers', 'Inflows of foreign population by nationality': 'migrants'})
    #correct country names - very slow, maybe refactor
    #oecd['country'] = oecd['country'].apply(corr_name)
    #oecd['destination'] = oecd['destination'].apply(corr_name)
    #set index
    oecd=oecd.set_index(['country', 'year'])
    return oecd

oecd_df=load_oecd_data()
oecd_df.info()
oecd_df.head(50)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 131072 entries, (Afghanistan, 2000) to (Zimbabwe, 2017)
Data columns (total 3 columns):
destination       131072 non-null object
asylum_seekers    83187 non-null float64
migrants          88009 non-null float64
dtypes: float64(2), object(1)
memory usage: 3.4+ MB


variable         destination  asylum_seekers  migrants
country     year                                      
Afghanistan 2000   Australia          1326.0     887.0
            2001   Australia          2161.0     456.0
            2002   Australia            53.0     660.0
            2003   Australia            54.0    1015.0
            2004   Australia           116.0    1340.0
            2005   Australia            32.0    3463.0
            2006   Australia            21.0    3465.0
            2007   Australia            20.0    2560.0
            2008   Australia            52.0    2033.0
            2009   Australia           940.0    1696.0
            2010   Australia          1265.0    1618.0
            2011   Australia          1720.0    2030.0
            2012   Australia          3064.0    1623.0
            2013   Australia           370.0    2178.0
            2014   Australia           123.0    5686.0
            2015   Australia           567.0    3323.0
            2016   Australia          2563.0    3197.0
            2017   Australia          1478.0    3566.0
            2000     Austria          4205.0     828.0
            2001     Austria         12955.0    1539.0
            2002     Austria          6651.0    1101.0
            2003     Austria          2357.0     743.0
            2004     Austria           757.0     650.0
            2005     Austria           923.0     652.0
            2006     Austria           699.0     498.0
            2007     Austria           761.0     530.0
            2008     Austria          1382.0     958.0
            2009     Austria          2237.0    1422.0
            2010     Austria          1582.0    1281.0
            2011     Austria          3609.0    2908.0
            2012     Austria          4003.0    3756.0
            2013     Austria          2589.0    2305.0
            2014     Austria          4916.0    3171.0
            2015     Austria         25143.0   19505.0
            2016     Austria         11506.0   11660.0
            2017     Austria          3525.0    2103.0
            2000     Belgium           861.0       NaN
            2001     Belgium           504.0       NaN
            2002     Belgium           326.0       NaN
            2003     Belgium           329.0       NaN
            2004     Belgium           287.0       NaN
            2005     Belgium           253.0       NaN
            2006     Belgium           365.0       NaN
            2007     Belgium           696.0       NaN
            2008     Belgium           879.0     138.0
            2009     Belgium          1659.0     150.0
            2010     Belgium          1124.0     208.0
            2011     Belgium          2774.0     285.0
            2012     Belgium          2349.0    2804.0
            2013     Belgium           892.0    1255.0

In [3]:
'''
Load & format Gross Domestic Product per Capita data
'''

def load_gdp_data():
    """ 
    Load gdp data file
    Reshape the dataset to have country, year, GDP columns
    Change data types
    Set hierarchical index (country, year)
    
    Returns
    --------
    gdp: data frame containing gdp data
    """
    #load dataset
    gdp = pd.read_csv('data/GDPPC_data.csv',na_values=['..'])
    #drop unwanted columns
    gdp.drop(['Indicator Name','Indicator Code','Country Code' ], inplace=True, axis=1)
    #reshape dataframe
    gdp.set_index(['Country Name'], inplace=True)
    gdp=gdp.stack().to_frame()
    gdp.reset_index(drop=False, inplace=True)
    #rename columns
    gdp=gdp.rename(columns={'Country Name': 'country', 'level_1': 'year', 0: 'GDP'})
    #set datatype for year, GDP
    gdp['year']=gdp['year'].astype(int)
    gdp['GDP']=gdp['GDP'].astype(float)
    #only use data from 2000 onwards
    gdp = gdp[gdp.year >= 2000]
    #correct country names
    #gdp['country'] = gdp['country'].apply(corr_name)
    #set index
    gdp=gdp.set_index(['country', 'year'])
    return gdp

gdp_df=load_gdp_data()
gdp_df.info()
gdp_df.tail(10)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4738 entries, (Aruba, 2000) to (Zimbabwe, 2018)
Data columns (total 1 columns):
GDP    4738 non-null float64
dtypes: float64(1)
memory usage: 53.2+ KB


GDP
country  year             
Zimbabwe 2009   771.598786
         2010   948.331854
         2011  1093.654002
         2012  1304.969802
         2013  1430.000818
         2014  1434.899340
         2015  1445.071062
         2016  1464.583529
         2017  1602.403507
         2018  2146.996385

In [4]:
'''
Load & format Human Development Index data
'''

def load_hdi_data():
    """ 
    Load hdi data file
    Reshape the dataset to have country, year, HDI columns
    Change data types
    Set hierarchical index (country, year)
    
    Returns
    --------
    hdi: data frame containing hdi data
    """
    #load dataset
    hdi = pd.read_csv('data/HDI.csv',na_values=['..'],)
    #drop unwanted columns
    hdi.drop(['HDI Rank (2018)'], inplace=True, axis=1)
    #reshape dataframe
    hdi.set_index(['Country'], inplace=True)
    hdi=hdi.stack().to_frame()
    hdi.reset_index(drop=False, inplace=True)
    #rename columns
    hdi=hdi.rename(columns={'Country': 'country', 'level_1': 'year', 0: 'HDI'})
    #set datatype for year
    hdi['year']=hdi['year'].astype(int)
    #only use data from 2000 onwards
    hdi = hdi[hdi.year >= 2000]
    #correct country names
    #hdi['country'] = hdi['country'].apply(corr_name)
    #set index
    hdi=hdi.set_index(['country', 'year'])
    return hdi

hdi_df=load_hdi_data()
hdi_df.info()
hdi_df.head(10)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3788 entries, (Afghanistan, 2000) to (World, 2018)
Data columns (total 1 columns):
HDI    3788 non-null float64
dtypes: float64(1)
memory usage: 42.6+ KB


HDI
country     year       
Afghanistan 2000  0.345
            2001  0.347
            2002  0.378
            2003  0.387
            2004  0.400
            2005  0.410
            2006  0.419
            2007  0.431
            2008  0.436
            2009  0.447

In [5]:
'''
Load & format World Governance Index data
'''

def load_wgi_data():
    """ 
    Load wgi data file
    Reshape the dataset to have country, year, WGI columns
    Change data types
    Set hierarchical index (country, year)
    
    Returns
    --------
    hdi: data frame containing hdi data
    """
    #load dataset
    wgi = pd.read_csv('data/WGIData.csv')
    #drop unwanted rows
    wgi=wgi[wgi['Indicator Code'].str.contains('EST', regex= True, na=False)]
    #drop unwanted columns
    wgi.drop(['Country Code','Indicator Name'], inplace=True, axis=1)
    #reshape dataframe
    wgi.set_index(['Country Name','Indicator Code'], inplace=True)
    wgi=wgi.stack().to_frame()
    wgi.reset_index(drop=False, inplace=True)
    #rename columns
    wgi=wgi.rename(columns={'Country Name': 'country', 'level_2': 'year', 0: 'variable'})
    wgi.set_index(['country','year','Indicator Code'], inplace=True)
    wgi=wgi.unstack()
    wgi.columns = wgi.columns.droplevel(0)
    wgi.reset_index(drop=False, inplace=True)
    #set datatype for year 
    wgi['year']=wgi['year'].astype(int)
    #only use data from 2000 onwards
    wgi = wgi[wgi.year >= 2000]
    #correct country names
    #wgi['country'] = wgi['country'].apply(corr_name)
    #set index
    wgi=wgi.set_index(['country', 'year'])
    return wgi
wgi_df=load_wgi_data()
wgi_df.info()
wgi_df.head(10)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3780 entries, (Afghanistan, 2000) to (Zimbabwe, 2018)
Data columns (total 6 columns):
CC.EST    3730 non-null float64
GE.EST    3720 non-null float64
PV.EST    3734 non-null float64
RL.EST    3769 non-null float64
RQ.EST    3719 non-null float64
VA.EST    3737 non-null float64
dtypes: float64(6)
memory usage: 190.2+ KB


Indicator Code      CC.EST    GE.EST    PV.EST    RL.EST    RQ.EST    VA.EST
country     year                                                            
Afghanistan 2000 -1.295380 -2.231651 -2.440187 -1.778437 -2.109626 -2.031417
            2002 -1.263366 -1.567973 -2.035389 -1.670823 -1.804449 -1.433421
            2003 -1.351042 -1.082488 -2.197809 -1.553829 -1.430002 -1.177571
            2004 -1.345281 -0.894051 -2.296179 -1.697143 -1.505863 -1.203034
            2005 -1.443609 -1.210829 -2.070934 -1.658668 -1.637369 -1.125430
            2006 -1.432890 -1.430343 -2.221302 -1.863189 -1.667902 -1.110294
            2007 -1.587331 -1.403352 -2.410581 -1.829666 -1.688257 -1.057654
            2008 -1.638287 -1.496089 -2.690477 -1.864380 -1.621583 -1.168902
            2009 -1.534796 -1.483899 -2.712689 -1.847092 -1.674311 -1.375497
            2010 -1.635723 -1.454683 -2.579152 -1.845436 -1.532861 -1.404468

## Country name inconsistency check before data merge
Before data merge we have to check the datasets for inconsistencies. 
We would like to merge on country and year multiindex.
Year is consistent however we have to search for different usage and typos in county names.
We selected oecd_df as base dataframe so we compare each of the entries in the country and destination columns to check whether they are also available in the hdi_df, gdp_df, wgi_df datasets.
### TODO - Inconsistencies have to be resolved by renaming country names!!!

In [6]:
def country_check(countries, gdp_df, hdi_df, wgi_df):
    """ 
    Parameters
    --------
    countries: list of countries (source or destination) in oecd_df
    gdp_df: gdp of the countries
    hdi_df: hdi of the countries
    wgi_df: wgi data of the countries
    
    Returns
    --------
    check: dataframe showing inconsistent country name usage
    """   
    #get country name list series
    hdi_c = hdi_df.index.unique(level=0).to_series()
    gdp_c = gdp_df.index.unique(level=0).to_series()
    wgi_c = wgi_df.index.unique(level=0).to_series()
    
    #create dataframe for the results
    check=pd.DataFrame(columns=['country','hdi','gdp','wgi'])
    #iterate through oecd_df source or destination country names and check presence of country name in other dfs
    for index,row in countries.iterrows():
        check=check.append({'country': row.country,
                            'hdi':row.isin(hdi_c).values[0],
                           'gdp':row.isin(gdp_c).values[0],
                           'wgi':row.isin(wgi_c).values[0]}, ignore_index=True)
        
    #add only problematic country rows to the result df
    check=check.loc[(check['hdi'] == False) | (check['gdp'] == False) | (check['wgi'] == False)]
    return check

oecd_s = pd.DataFrame(oecd_df.index.unique(level=0))
oecd_s_check = country_check(oecd_s,gdp_df, hdi_df, wgi_df)

oecd_d = pd.DataFrame(oecd_df['destination'].unique(),columns={'country'})
oecd_d_check = country_check(oecd_d,gdp_df, hdi_df, wgi_df)

display(oecd_d_check.head(50),oecd_s_check.head(50))
df = pd.concat([oecd_d_check,oecd_s_check])
#df['country']

,country,hdi,gdp,wgi
5,Czech Republic,False,True,True
18,Korea,False,False,False
27,Slovak Republic,False,True,True


,country,hdi,gdp,wgi
11,Bahamas,True,False,False
19,Bermuda,False,True,True
21,Bolivia,False,True,True
32,Cape Verde,False,False,False
37,Chinese Taipei,False,False,False
40,Congo,True,False,False
41,Cook Islands,False,False,True
46,Czech Republic,False,True,True
47,Côte d'Ivoire,True,False,False
48,Democratic People's Republic of Korea,False,False,False


We can use the pycountry package to lookup country names using the search_fuzzy function.
The given country name will be replaced by the common_name variable from the results list

In [7]:
def country_correction(oecd_df, gdp_df, hdi_df, wgi_df):
    """ 
    Parameters
    --------
    oecd_df: problematic source country names
    gdp_df: gdp of the countries
    hdi_df: hdi of the countries
    wgi_df: wgi data of the countries
    
    Returns
    --------
    oecd_df_corr: merged data frame that contains corrected country names in the datasets
    """
    
    #replace country name with common_name in each files
    #for row in oecd_df.iterrows():
    #    [name for name in row[0]]
        
        
    
    

country_correction(oecd_df, gdp_df, hdi_df, wgi_df)

In [8]:
def merge_dest(s_merged, gdp_df, hdi_df, wgi_df):
    """ 
    Parameters
    --------
    s_merged: merged dataset with values for the source country
    gdp_df: gdp of the countries
    hdi_df: hdi of the countries
    wgi_df: wgi data of the countries
    
    Returns
    --------
    d_merged: merged data frame that contains complete migration and country data for the source and destination countries
    """
    s_merged = s_merged.rename(columns={'destination': 'country'})
    s_merged=s_merged.set_index(['year','country'])
    #concatenate gdp_df, hdi_df, wgi_df, s_merged using year, country keys
    merge1=pd.concat([gdp_df, hdi_df], axis=1, keys=['year', 'country'])
    merge1.columns = merge1.columns.droplevel(0)
    merge2=pd.concat([merge1, wgi_df], axis=1, keys=['year', 'country'])
    merge2.columns = merge2.columns.droplevel(0)
    d_merged=pd.merge(merge2,s_merged,on=['year','country'])
    d_merged=d_merged.reset_index()
    d_merged = d_merged.rename(columns={'GDP': 'd_GDP',
                                             'HDI': 'd_HDI',
                                             'CC.EST': 'd_CC.ESTP',
                                             'GE.EST': 'd_GE.EST',
                                             'PV.EST': 'd_PV.EST',
                                             'RL.EST': 'd_RL.EST',
                                             'VA.EST': 'd_VA.EST',
                                             'country': 'destination'})
    return d_merged


def merge_data(oecd_df, gdp_df, hdi_df, wgi_df):
    """ 
    Parameters
    --------
    oecd_df: yearly data for foreign population inflow and asylum seeker inflow from source to destination country
    gdp_df: gdp of the countries
    hdi_df: hdi of the countries
    wgi_df: wgi data of the countries
    
    Returns
    --------
    merged_data: merged data frame that contains complete migration and country data
    """
    #concatenate gdp_df, hdi_df, wgi_df, oecd_df using year, country keys
    merge1=pd.concat([gdp_df, hdi_df], axis=1, keys=['year', 'country'])
    merge1.columns = merge1.columns.droplevel(0)
    merge2=pd.concat([merge1, wgi_df], axis=1, keys=['year', 'country'])
    merge2.columns = merge2.columns.droplevel(0)
    s_merged=pd.merge(merge2,oecd_df,on=['year','country'])
    s_merged=s_merged.reset_index()
    s_merged = s_merged.rename(columns={'GDP': 's_GDP',
                                             'HDI': 's_HDI',
                                             'CC.EST': 's_CC.ESTP',
                                             'GE.EST': 's_GE.EST',
                                             'PV.EST': 's_PV.EST',
                                             'RL.EST': 's_RL.EST',
                                             'VA.EST': 's_VA.EST',
                                             'country': 'source'})
    merged_data=merge_dest(s_merged,gdp_df, hdi_df, wgi_df)
    #set multiindex with year, source, destination
    merged_data=merged_data.set_index(['year','source','destination'])
    
    return merged_data

data_merged = merge_data(oecd_df, gdp_df, hdi_df, wgi_df)
data_merged.to_csv("merged.csv")
#data_merged.info()
data_merged

d_GDP  d_HDI  d_CC.ESTP  d_GE.EST  \
year source      destination                                              
2000 Afghanistan Australia      21679.24784  0.898   1.923246  1.810187   
     Albania     Australia      21679.24784  0.898   1.923246  1.810187   
     Algeria     Australia      21679.24784  0.898   1.923246  1.810187   
     Andorra     Australia      21679.24784  0.898   1.923246  1.810187   
     Angola      Australia      21679.24784  0.898   1.923246  1.810187   
...                                     ...    ...        ...       ...   
2017 Vanuatu     United States  59927.92983  0.919   1.381242  1.554298   
     Viet Nam    United States  59927.92983  0.919   1.381242  1.554298   
     Yemen       United States  59927.92983  0.919   1.381242  1.554298   
     Zambia      United States  59927.92983  0.919   1.381242  1.554298   
     Zimbabwe    United States  59927.92983  0.919   1.381242  1.554298   

                                d_PV.EST  d_RL.EST  RQ.EST_x  d_VA.EST  \
year source      destination                                             
2000 Afghanistan Australia      1.334552  1.751407  1.637249  1.471305   
     Albania     Australia      1.334552  1.751407  1.637249  1.471305   
     Algeria     Australia      1.334552  1.751407  1.637249  1.471305   
     Andorra     Australia      1.334552  1.751407  1.637249  1.471305   
     Angola      Australia      1.334552  1.751407  1.637249  1.471305   
...                                  ...       ...       ...       ...   
2017 Vanuatu     United States  0.335553  1.644761  1.628115  1.049652   
     Viet Nam    United States  0.335553  1.644761  1.628115  1.049652   
     Yemen       United States  0.335553  1.644761  1.628115  1.049652   
     Zambia      United States  0.335553  1.644761  1.628115  1.049652   
     Zimbabwe    United States  0.335553  1.644761  1.628115  1.049652   

                                       s_GDP  s_HDI  s_CC.ESTP  s_GE.EST  \
year source      destination                                               
2000 Afghanistan Australia               NaN  0.345  -1.295380 -2.231651   
     Albania     Australia       1126.683318  0.667  -0.857256 -0.755118   
     Algeria     Australia       1764.888222  0.646  -0.937911 -0.963841   
     Andorra     Australia      21936.530100  0.759   1.359712  1.446284   
     Angola      Australia        556.836318  0.394  -1.522685 -1.462310   
...                                      ...    ...        ...       ...   
2017 Vanuatu     United States   3082.356323  0.595   0.005782 -0.901097   
     Viet Nam    United States           NaN  0.690        NaN       NaN   
     Yemen       United States           NaN  0.463        NaN       NaN   
     Zambia      United States   1534.865371  0.589  -0.542213 -0.627568   
     Zimbabwe    United States   1602.403507  0.553  -1.273584 -1.188722   

                                s_PV.EST  s_RL.EST  RQ.EST_y  s_VA.EST  \
year source      destination                                             
2000 Afghanistan Australia     -2.440187 -1.778437 -2.109626 -2.031417   
     Albania     Australia     -0.537979 -1.008780 -0.254359 -0.285139   
     Algeria     Australia     -1.432010 -1.211336 -0.705386 -1.110052   
     Andorra     Australia      1.167002  1.312131  1.276005  1.535655   
     Angola      Australia     -2.035583 -1.663307 -1.799331 -1.458065   
...                                  ...       ...       ...       ...   
2017 Vanuatu     United States  0.701113  0.487920 -0.331086  0.693794   
     Viet Nam    United States       NaN       NaN       NaN       NaN   
     Yemen       United States       NaN       NaN       NaN       NaN   
     Zambia      United States  0.146180 -0.325046 -0.466104 -0.335299   
     Zimbabwe    United States -0.708677 -1.377855 -1.563330 -1.196420   

                                asylum_seekers  migrants  
year source      destination                              
2000 Afghanistan Australia   

In [9]:
def handle_missing(df):
    '''
    Parameters
    --------
    df: dataframe with missing values to be filled
    
    Returns
    --------
    df: dataframe with no missing values
    '''
    #reset index to interpolate
    df.reset_index(inplace=True)
    
    #display(df.loc[df.destination=='Australia'].loc[df.source=='Zambia'])
    
    #group data by source and dest, so we only interpolate on subsets of data instead of whole frame
    df = df.groupby(['destination','source']).apply(
        lambda group: group.interpolate(method='linear',limit_direction='both')) #enter interpolation method here
    
    #display(df.loc[df.destination=='Australia'].loc[df.source=='Zambia'])
    
    df.set_index(['destination','source','year'], inplace=True)
    
    return df

data_complete = handle_missing(data_merged)
data_complete.info()
data_complete

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 112880 entries, (Australia, Afghanistan, 2000) to (United States, Zimbabwe, 2017)
Data columns (total 18 columns):
d_GDP             112880 non-null float64
d_HDI             106240 non-null float64
d_CC.ESTP         112880 non-null float64
d_GE.EST          112880 non-null float64
d_PV.EST          112880 non-null float64
d_RL.EST          112880 non-null float64
RQ.EST_x          112880 non-null float64
d_VA.EST          112880 non-null float64
s_GDP             105740 non-null float64
s_HDI             104482 non-null float64
s_CC.ESTP         105094 non-null float64
s_GE.EST          105094 non-null float64
s_PV.EST          106318 non-null float64
s_RL.EST          106318 non-null float64
RQ.EST_y          105094 non-null float64
s_VA.EST          106318 non-null float64
asylum_seekers    108858 non-null float64
migrants          96587 non-null float64
dtypes: float64(18)
memory usage: 15.9+ MB


d_GDP  d_HDI  d_CC.ESTP  d_GE.EST  \
destination   source      year                                            
Australia     Afghanistan 2000  21679.24784  0.898   1.923246  1.810187   
              Albania     2000  21679.24784  0.898   1.923246  1.810187   
              Algeria     2000  21679.24784  0.898   1.923246  1.810187   
              Andorra     2000  21679.24784  0.898   1.923246  1.810187   
              Angola      2000  21679.24784  0.898   1.923246  1.810187   
...                                     ...    ...        ...       ...   
United States Vanuatu     2017  59927.92983  0.919   1.381242  1.554298   
              Viet Nam    2017  59927.92983  0.919   1.381242  1.554298   
              Yemen       2017  59927.92983  0.919   1.381242  1.554298   
              Zambia      2017  59927.92983  0.919   1.381242  1.554298   
              Zimbabwe    2017  59927.92983  0.919   1.381242  1.554298   

                                d_PV.EST  d_RL.EST  RQ.EST_x  d_VA.EST  \
destination   source      year                                           
Australia     Afghanistan 2000  1.334552  1.751407  1.637249  1.471305   
              Albania     2000  1.334552  1.751407  1.637249  1.471305   
              Algeria     2000  1.334552  1.751407  1.637249  1.471305   
              Andorra     2000  1.334552  1.751407  1.637249  1.471305   
              Angola      2000  1.334552  1.751407  1.637249  1.471305   
...                                  ...       ...       ...       ...   
United States Vanuatu     2017  0.335553  1.644761  1.628115  1.049652   
              Viet Nam    2017  0.335553  1.644761  1.628115  1.049652   
              Yemen       2017  0.335553  1.644761  1.628115  1.049652   
              Zambia      2017  0.335553  1.644761  1.628115  1.049652   
              Zimbabwe    2017  0.335553  1.644761  1.628115  1.049652   

                                       s_GDP  s_HDI  s_CC.ESTP  s_GE.EST  \
destination   source      year                                             
Australia     Afghanistan 2000    179.426495  0.345  -1.295380 -2.231651   
              Albania     2000   1126.683318  0.667  -0.857256 -0.755118   
              Algeria     2000   1764.888222  0.646  -0.937911 -0.963841   
              Andorra     2000  21936.530100  0.759   1.359712  1.446284   
              Angola      2000    556.836318  0.394  -1.522685 -1.462310   
...                                      ...    ...        ...       ...   
United States Vanuatu     2017   3082.356323  0.595   0.005782 -0.901097   
              Viet Nam    2017           NaN  0.690        NaN       NaN   
              Yemen       2017           NaN  0.463        NaN       NaN   
              Zambia      2017   1534.865371  0.589  -0.542213 -0.627568   
              Zimbabwe    2017   1602.403507  0.553  -1.273584 -1.188722   

                                s_PV.EST  s_RL.EST  RQ.EST_y  s_VA.EST  \
destination   source      year                                           
Australia     Afghanistan 2000 -2.440187 -1.778437 -2.109626 -2.031417   
              Albania     2000 -0.537979 -1.008780 -0.254359 -0.285139   
              Algeria     2000 -1.432010 -1.211336 -0.705386 -1.110052   
              Andorra     2000  1.167002  1.312131  1.276005  1.535655   
              Angola      2000 -2.035583 -1.663307 -1.799331 -1.458065   
...                                  ...       ...       ...       ...   
United States Vanuatu     2017  0.701113  0.487920 -0.331086  0.693794   
              Viet Nam    2017       NaN       NaN       NaN       NaN   
              Yemen       2017       NaN       NaN       NaN       NaN   
              Zambia      2017  0.146180 -0.325046 -0.466104 -0.335299   
              Zimbabwe    2017 -0.708677 -1.377855 -1.563330 -1.196420   

                                asylum_seekers  migrants  
destination   source      year                            
Australia     Afghanistan 200